In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1934171/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_2,0.809041,0.273258,0.657467,0.932168,0.863713,0.358985,4.859726,0.373912,0.203379,0.288645,1.471138,0.599154,1.707671,0.608741,162.048948,334.759595,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_9_1,0.809023,0.273143,0.674995,0.937381,0.871920,0.359020,4.860496,0.354777,0.187750,0.271264,1.477427,0.599183,1.707739,0.608770,162.048756,334.759402,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_9_3,0.808497,0.273341,0.639515,0.925079,0.854069,0.360008,4.859172,0.393508,0.224635,0.309071,1.464979,0.600007,1.709687,0.609607,162.043258,334.753905,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_9_0,0.808467,0.272970,0.691831,0.940925,0.878768,0.360064,4.861654,0.336399,0.177122,0.256761,1.483893,0.600053,1.709798,0.609655,162.042946,334.753593,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_9_4,0.807374,0.273410,0.621416,0.915944,0.842939,0.362119,4.858708,0.413265,0.252023,0.332643,1.458883,0.601763,1.713849,0.611392,162.031564,334.742211,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,model_1_2_1,0.735264,0.389371,0.123548,0.410725,0.305564,0.497679,4.083278,1.256390,0.784731,1.020561,2.228233,0.705464,1.981080,0.716752,161.395599,334.106246,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
155,model_1_6_20,0.734943,0.406306,0.067139,0.621937,0.525800,0.498284,3.970033,0.976133,1.866911,1.421521,2.164998,0.705892,1.982271,0.717187,161.393171,334.103818,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
156,model_1_2_23,0.734724,0.391137,-0.015486,0.249158,0.164555,0.498694,4.071472,1.455695,0.999888,1.227791,2.103637,0.706183,1.983081,0.717483,161.391524,334.102171,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
157,model_1_2_0,0.734442,0.389095,0.127699,0.416255,0.310094,0.499225,4.085122,1.250440,0.777366,1.013903,2.236621,0.706559,1.984128,0.717864,161.389395,334.100042,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
